In [129]:
!nvidia-smi # kill -9 -1(모두 종료)

Sun Dec 24 01:09:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 537.34       CUDA Version: 12.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 50%   27C    P8    32W / 420W |  13965MiB / 24576MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [130]:
!kill -9 72225

: 

In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

/home/wsl_han/anaconda3/envs/aivle_project/lib/python3.10/site-packages/transformers/utils/hub.py:122: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
model_name_or_path = "TheBloke/Llama-2-13b-Chat-GPTQ"#"mistralai/Mistral-7B-v0.1"  # mistralai/Mistral-7B-Instruct-v0.1, meta-llama/Llama-2-13b-chat-hf
 
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [56]:
# Use a pipeline for later
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 5,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [57]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")



def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [58]:
instruction = "User: {user_input}"
system_prompt = "You must distinguish among South Korea's mobile telecommunication companies (skt, kt, lg u+). Provide answers with ONLY 'yes' or 'no'. no more answer"

template = get_prompt(instruction, system_prompt)
print(template)

[INST]<<SYS>>
You must distinguish among South Korea's mobile telecommunication companies (skt, kt, lg u+). Provide answers with ONLY 'yes' or 'no'. no more answer
<</SYS>>

User: {user_input}[/INST]


In [59]:
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate
from langchain import HuggingFacePipeline

In [60]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

In [61]:
prompt = PromptTemplate(
    input_variables=["user_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [62]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    # memory=memory,
)

In [127]:
a = llm_chain.predict(user_input="'kt인터넷 쓰는데 가끔은 게임할때 끊기고 그럼. 그래서 고객센터 전화하면 연결 잘 안되더라..' inputed text related KT??")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You must distinguish among South Korea's mobile telecommunication companies (skt, kt, lg u+). Provide answers with ONLY 'yes' or 'no'. no more answer
<</SYS>>

User: 'kt인터넷 쓰는데 가끔은 게임할때 끊기고 그럼. 그래서 고객센터 전화하면 연결 잘 안되더라..' inputed text related KT??[/INST]


/home/wsl_han/anaconda3/envs/aivle_project/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


In [128]:
print(a)

  Yes.


In [87]:
import pandas as pd
import time

In [114]:
data = pd.read_csv('./kt.csv')

In [20]:
data

,Unnamed: 0,comment,title,date
0,0,처음 이사 들어설때 공유기 없어서 내돈내산으로 설치함 일단 1번더 끊기면 kt에 전...,집주인한테 뭐라 말해야 인터넷 개별설치 허락해줄까,2023-12-22 03:24
1,1,아 맞다 ㅅㅂ 그리고 계약할때 인터넷 이미 돼있다고 kt 쓰라고 했는데 안되면 이것...,집주인한테 뭐라 말해야 인터넷 개별설치 허락해줄까,2023-12-22 03:16
2,2,카톡으로 최근들어 인터넷 끊김이 자주 발생해서 kt 고객센터에 신호 다시 받거나 기...,집주인한테 뭐라 말해야 인터넷 개별설치 허락해줄까,2023-12-22 03:11
3,3,"난 3인가족다kt, kt티비, 인터넷 써서 결합하니까 알뜰폰이랑 거의 차이 없고 오...",폰 자급제는 절대 사지 말라더라..jpg,2023-12-21 22:42
4,4,학교 인터넷이라서 KT인걸로 알고 있음,지금 끊기는 사람들 통신사 설문 한번 해보자,2023-12-21 20:41
...,...,...,...,...
995,995,멀티연결 하는게임들을 국산게임 위주로 하시는거면 SK도 사실 아무상관없습니다. 문제...,인터넷 skt kt 질문드립니다,2023-07-02 10:24
996,996,KT가 보편적으로 빠르긴한데 내가 느끼기는 인터넷은 그 지역에 잘 터지는게있는거같음,인터넷 속도는 KT가 1등임?,2023-07-02 09:24
997,997,혹시 모르니 speed.kt.com으로 인터넷 명의자 아이디로 로그인 후 SLA 테...,kt인터넷 속도,2023-07-01 19:24
998,998,방을 옮겼더니 인터넷 연결이 안잡혀서요 kt 기사 불렀더니 기사 몇 분이 코로나 걸...,무선랜카드 질문,2023-06-30 15:45


In [21]:
filtered_list = []
not_filtered_list = []

In [ ]:
filtered_list = []
not_filtered_list = []

total_start = time.time()
for comment in data['comment'].values[:300]:
    start = time.time()
    result = llm_chain.predict(user_input=f"'{comment}'입력된 텍스트는 kt에 대한 내용이야?")
    end = time.time() - start
    
    if (result.strip() == 'Yes') or (result.strip() == 'Yes.'):
        filtered_list.append((comment, end, result.strip()))
    else:
        not_filtered_list.append((comment, end, result.strip()))

total_end = time.time() - total_start

pd.DataFrame({'commnet':filtered_list}).to_csv('./filtering_llama2_13B_kt_300.csv',index=False)

pd.DataFrame({'commnet':not_filtered_list}).to_csv('./not_filtering_13B_llama2_kt_300.csv',index=False)

In [82]:
filtered_list

[('처음 이사 들어설때 공유기 없어서 내돈내산으로 설치함 일단 1번더 끊기면 kt에 전화해서 기사님 모시고 단자함이나 여러 설명 드려봐야겠네 그리고 2층이 인터넷 끊기면 3층도 끊기던데 왜그러지',
  0.4521183967590332,
  'Yes'),
 ('아 맞다 ㅅㅂ 그리고 계약할때 인터넷 이미 돼있다고 kt 쓰라고 했는데 안되면 이것도 말해야겠네 믿었는데 끊김 심하다고',
  0.23281574249267578,
  'Yes'),
 ('카톡으로 최근들어 인터넷 끊김이 자주 발생해서 kt 고객센터에 신호 다시 받거나 기사분들 몇명이 오셨는데도 해결이 안됐다 혹시 인터넷 해결방안이 있는지 물어보고 그다음 개별설치 물어보는데 안되면 ㅈㅈ긴함 감정싸움 일어나면 나중에 끝나고 이사할때 보증금이나 돈 관련으로 빡세지는거 나도 잘 알아서',
  0.2508828639984131,
  'Yes'),
 ('학교 인터넷이라서 KT인걸로 알고 있음', 0.22951221466064453, 'Yes'),
 ('나도 없 인터넷 문젠가 본인kt', 0.2330782413482666, 'Yes'),
 ('다음 폰 바꿀 시점엔 자급제도 알아보긴해야겠네요 근데 인터넷이랑 가족 5명이 kt가족할인으로 묶여있어서 쉽지는 않을듯요...ㅋㅋㅋ',
  0.2169342041015625,
  'Yes'),
 ('인터넷은 kt 임', 0.1445157527923584, 'Yes'),
 ('외워라 통신사는sk 인터넷은kt', 0.13665509223937988, 'Yes'),
 ('아 나는 kt인데 최근에 인터넷이 느려가지고ㅋㅋ sk면 뭐지', 0.13767051696777344, 'Yes'),
 ('원래 인터넷은 kt였는데 요샌 진짜 이상하네여...', 0.13467001914978027, 'Yes'),
 ('예전부터 인터넷문제잇던데 왜안바꾸지 건물이 kt밖에못쓰나??', 0.13248753547668457, 'Yes'),
 ('인터넷은 무조건 KT 라고 생각함', 0.175998687744

In [27]:
pd.DataFrame({'commnet':filtered_list}).to_csv('./filtering_llama2_13B_kt_300.csv',index=False)


In [28]:
pd.DataFrame({'commnet':not_filtered_list}).to_csv('./not_filtering_13B_llama2_kt_300.csv',index=False)


In [30]:
!transformers-cli cache clear

usage: transformers-cli <command> [<args>]
Transformers CLI tool: error: argument {convert,download,env,run,serve,login,whoami,logout,repo,add-new-model,add-new-model-like,lfs-enable-largefiles,lfs-multipart-upload,pt-to-tf}: invalid choice: 'cache' (choose from 'convert', 'download', 'env', 'run', 'serve', 'login', 'whoami', 'logout', 'repo', 'add-new-model', 'add-new-model-like', 'lfs-enable-largefiles', 'lfs-multipart-upload', 'pt-to-tf')
